# Why Julia?

"We want a language that’s open source, with a liberal license. We want the speed of C with the dynamism of Ruby. We want a language that’s homoiconic, with true macros like Lisp, but with obvious, familiar mathematical notation like Matlab. We want something as usable for general programming as Python, as easy for statistics as R, as natural for string processing as Perl, as powerful for linear algebra as Matlab, as good at gluing programs together as the shell. Something that is dirt simple to learn, yet keeps the most serious hackers happy. We want it interactive and we want it compiled.

(Did we mention it should be as fast as C?)"

Basically they want the moon on a stick. They're pretty close to getting it.

## This tutorial

This is a whirlwind tour of Julia. We'll cover:
- Data types
- Basic control flow
- Functions 
- Linear algebra
- Introspection
- Interoperability
- Parallelisation

## Data types

Default numeric type is double precision: Float64. Also supports, Float32, Int64, etc

In [132]:
one(Float64)

1.0

In [137]:
rand(Float32)

0.8634758f0

In [135]:
zero(Int64)

0

Division causes upgrade to Float64. This allows type stability in programs.

In [124]:
1/3

0.3333333333333333

Also fancy things like rational numbers.

In [112]:
r = 3//7 * 7//5

3//5

In [139]:
typeof(r)

Rational{Int64}

And unlimited precision arithmetic


In [155]:
factorial(200) #woops - this is a big number

LoadError: LoadError: OverflowError()
while loading In[155], in expression starting on line 1

In [151]:
n = 200*one(BigInt)

200

In [152]:
typeof(n)

BigInt

In [153]:
factorial(n)

788657867364790503552363213932185062295135977687173263294742533244359449963403342920304284011984623904177212138919638830257642790242637105061926624952829931113462857270763317237396988943922445621451664240254033291864131227428294853277524242407573903240321257405579568660226031904170324062351700858796178922222789623703897374720000000000000000000000000000000000000000000000000

Tuples

In [130]:
x = (1,0,"bob")

(1,0,"bob")

In [131]:
collect(enumerate(x))

3-element Array{Tuple{Int64,Any},1}:
 (1,1)    
 (2,0)    
 (3,"bob")

Dictionaries

In [125]:
y = Dict("key"=>"value", :foo=>2, 3=>"baz")

Dict{Any,Any} with 3 entries:
  "key" => "value"
  :foo  => 2
  3     => "baz"

In [127]:
keys(y)

Base.KeyIterator for a Dict{Any,Any} with 3 entries. Keys:
  "key"
  :foo
  3

In [128]:
values(y)

Base.ValueIterator for a Dict{Any,Any} with 3 entries. Values:
  "value"
  2
  "baz"

Vectors

In [60]:
v = zeros(5)

5-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [28]:
length(v)

5

In [119]:
l = collect(1:length(v))

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

Matrices are column major

In [90]:
M = reshape(collect(1:25),5,5)

5×5 Array{Int64,2}:
 1   6  11  16  21
 2   7  12  17  22
 3   8  13  18  23
 4   9  14  19  24
 5  10  15  20  25

In [120]:
size(M)

(5,5)

In [121]:
typeof(M)

Array{Int64,2}

## Control flow

In [65]:
for i in 1:length(v)
    if i > 2
        v[i] = sqrt(i)
    elseif i==1
        v[i] = -1
    else
        v[i] = 0
    end
end

In [66]:
println(v)

[-1.0,0.0,1.73205,2.0,2.23607]


## Linear algebra

Matrix operations are default: 
- multiplication *
- arithmetic +, -
- solve \

In [92]:
B = M' * M

5×5 Array{Int64,2}:
  55  130   205   280   355
 130  330   530   730   930
 205  530   855  1180  1505
 280  730  1180  1630  2080
 355  930  1505  2080  2655

For elementwise operations, add a . before the operator.

In [94]:
H = M .* M

5×5 Array{Int64,2}:
  1   36  121  256  441
  4   49  144  289  484
  9   64  169  324  529
 16   81  196  361  576
 25  100  225  400  625

Comes packed with blas and suitesparse out of the box.

In [73]:
C = chol(B)

5×5 UpperTriangular{Float64,Array{Float64,2}}:
 1.49465  0.704378  -0.319276   0.846436   0.702648
  ⋅       1.1001    -1.01232   -0.100628   0.364213
  ⋅        ⋅         1.46891   -0.539081  -1.29114 
  ⋅        ⋅          ⋅         1.14652    0.391745
  ⋅        ⋅          ⋅          ⋅         1.33747 

In [88]:
C \ l

5-element Array{Float64,1}:
 -4.06067
  6.43257
  6.13986
  2.21148
  3.73839

## Functions

In [37]:
function dostuff(N)
    v = zeros(N)
    for i in 1:N
        v[i] = sqrt(i)
    end
    return v
end

dostuff (generic function with 1 method)

In [40]:
dostuff(5)

5-element Array{Float64,1}:
 1.0    
 1.41421
 1.73205
 2.0    
 2.23607

In [42]:
sum(dostuff(5))

8.382332347441762

In [84]:
@elapsed dostuff(500000)

0.005578655

Introspect on your code with macros, including *inter alia* :
 - @code_warntype what types are getting used in your function, great for performance tweaking.
 - @code_native the machine code in case you don't believe it's actually a compiled language...

In [83]:
@code_warntype dostuff(3)

Variables:
  #self#::#dostuff
  N::Int64
  v::Array{Float64,1}
  #temp#::Int64
  i::Int64

Body:
  begin 
      v::Array{Float64,1} = $(Expr(:invoke, LambdaInfo for fill!(::Array{Float64,1}, ::Float64), :(Base.fill!), :((Core.ccall)(:jl_alloc_array_1d,(Core.apply_type)(Core.Array,Float64,1)::Type{Array{Float64,1}},(Core.svec)(Core.Any,Core.Int)::SimpleVector,Array{Float64,1},0,N,0)::Array{Float64,1}), :((Base.box)(Float64,(Base.sitofp)(Float64,0))))) # line 3:
      SSAValue(4) = (Base.select_value)((Base.sle_int)(1,N::Int64)::Bool,N::Int64,(Base.box)(Int64,(Base.sub_int)(1,1)))::Int64
      #temp#::Int64 = 1
      5: 
      unless (Base.box)(Base.Bool,(Base.not_int)((#temp#::Int64 === (Base.box)(Int64,(Base.add_int)(SSAValue(4),1)))::Bool)) goto 16
      SSAValue(5) = #temp#::Int64
      SSAValue(6) = (Base.box)(Int64,(Base.add_int)(#temp#::Int64,1))
      i::Int64 = SSAValue(5)
      #temp#::Int64 = SSAValue(6) # line 4:
      SSAValue(2) = (Base.Math.box)(Base.Math.Float64,(Base.Mat

## Probability distributions

In [13]:
using Distributions

In [9]:
f = Distributions.Laplace(0,1)

Distributions.Laplace(μ=0.0, θ=1.0)

In [10]:
mean(f)

0.0

In [11]:
var(f)

2.0

In [12]:
cdf(f, 1.65)

0.903975045689623

In [156]:
rand(f, 2, 5)

2×5 Array{Float64,2}:
 -1.04038  -4.73578   -1.19704  -0.314988   0.381115
 -0.23622  -0.168321  -1.13565   0.410258  -0.815807

## Metaprogramming

Code is data, and can be operated on and changed by other code. 


Before they get executed, your instructions are parsed. We can hang on to the parsed-but-not-yet-executed instructions with `Expr` objects.

In [176]:
somecode = :(M*M)

:(M * M)

In [183]:
typeof(somecode)

Expr

We can evaluate the instruction at a later point.

In [178]:
eval(somecode)

5×5 Array{Int64,2}:
 215  490   765  1040  1315
 230  530   830  1130  1430
 245  570   895  1220  1545
 260  610   960  1310  1660
 275  650  1025  1400  1775

In [181]:
M = 7*M

5×5 Array{Int64,2}:
  98  588  1078  1568  2058
 196  686  1176  1666  2156
 294  784  1274  1764  2254
 392  882  1372  1862  2352
 490  980  1470  1960  2450

In [184]:
eval(somecode)

5×5 Array{Int64,2}:
 2064860  4705960  7347060   9988160  12629260
 2208920  5090120  7971320  10852520  13733720
 2352980  5474280  8595580  11716880  14838180
 2497040  5858440  9219840  12581240  15942640
 2641100  6242600  9844100  13445600  17047100

We can even alter the behaviour of not-yet-executed code with `macro`s.

In [174]:
fieldnames(typeof(somecode))

3-element Array{Symbol,1}:
 :head
 :args
 :typ 

In [185]:
somecode.args

3-element Array{Any,1}:
 :*
 :M
 :M

For example, we could switch out matrix multiplication for elementwise multiplication.

In [162]:
macro corruptmatmult(arg)
   if arg.args[1] == :*
       arg.args[1] = :.*
   end
   return arg
end

@corruptmatmult (macro with 1 method)

In [165]:
M*M

5×5 Array{Int64,2}:
 215  490   765  1040  1315
 230  530   830  1130  1430
 245  570   895  1220  1545
 260  610   960  1310  1660
 275  650  1025  1400  1775

In [186]:
@corruptmatmult M*M

5×5 Array{Int64,2}:
   9604  345744  1162084  2458624  4235364
  38416  470596  1382976  2775556  4648336
  86436  614656  1623076  3111696  5080516
 153664  777924  1882384  3467044  5531904
 240100  960400  2160900  3841600  6002500

# Parallel 

Were you just thinking, "Sure, but why would I be interested in metaprogramming?" Fret no longer. 

Let's grab some more processors.

In [191]:
pids = addprocs(4)

4-element Array{Int64,1}:
 14
 15
 16
 17

In [193]:
workers()

4-element Array{Int64,1}:
 14
 15
 16
 17

In [224]:
@fetchfrom pids[2] myid()

15

In [228]:
[@fetchfrom pid randn() for pid in pids]

4-element Array{Float64,1}:
  1.14113  
 -0.0884254
  0.495689 
  0.256173 

In [238]:
@everywhere n = myid()

In [241]:
(myid(), pids[2])

(1,15)

In [240]:
@fetchfrom pids[2] n

1

Huh? We want to retrieve the value we previously assigned to the symbol `:n` on the second processor.

In [236]:
@fetchfrom pids[2] eval(:n)

15

So we need metaprogramming to do parallel computing conveniently.

I made a small package, `ClusterUtils`, to facilitate this.

In [201]:
using ClusterUtils

In [216]:
results = reap(pids, :(myid()*[1,2,3]))

Dict{Int64,Any} with 4 entries:
  14 => [14,28,42]
  16 => [16,32,48]
  17 => [17,34,51]
  15 => [15,30,45]

In [222]:
reduce(vcat, values(results))

12-element Array{Int64,1}:
 14
 28
 42
 16
 32
 48
 17
 34
 51
 15
 30
 45

In [242]:
reap(pids, :(x = randn(3)))

Dict{Int64,Any} with 4 entries:
  14 => [0.2691,0.749462,-0.740908]
  16 => [0.160716,-0.948511,0.513451]
  17 => [0.272587,-0.145759,-0.394778]
  15 => [0.280709,-0.0366645,0.928021]

In [244]:
reap(pids, :(pi*x))

Dict{Int64,Any} with 4 entries:
  14 => [0.845403,2.3545,-2.32763]
  16 => [0.504904,-2.97983,1.61305]
  17 => [0.856357,-0.457914,-1.24023]
  15 => [0.881873,-0.115185,2.91546]

## Interoperability

We can use the shell from Julia.

In [270]:
println(readstring(`cat /proc/meminfo`)[1:200] * "...")

MemTotal:       32425532 kB
MemFree:        26020776 kB
Buffers:            4704 kB
Cached:           756692 kB
SwapCached:        45684 kB
Active:          2525416 kB
Inactive:        1144516 kB
Acti...


We can use python modules from Julia.

In [245]:
using PyCall
@pyimport pylab

In [250]:
pylab.plot(map(sin, linspace(0,2pi,1000)))
pylab.show()

We can use C and Fortran libraries from Julia.

In [252]:
t = ccall( (:clock, "libc"), Int32, ())

57460000